In [7]:
from gensim.models import KeyedVectors
import pandas as pd
import re


In [8]:

# Remplacez "chemin/vers/glove.6B.50d.txt" par le chemin de votre fichier GloVe
chemin_glove = "glove.6B/glove.6B.50d.txt"


In [9]:
import numpy as np

embeddings_dict = {}
with open(chemin_glove) as f:
    for line in f:
        values = line.split()
        word = values[0]
        vector = np.asarray(values[1:], "float32")
        embeddings_dict[word] = vector

In [10]:
df = pd.read_csv("most_frequent_words.csv")
df


,word,count,en,sentiment
0,tres,12626,Very,POSITIVE
1,vehicule,8204,Vehicle,MIXED
2,voiture,4936,Car,POSITIVE
3,livraison,4556,Delivery,MIXED
4,bien,4415,Well,MIXED
...,...,...,...,...
995,vacances,80,Holidays,MIXED
996,rappeler,80,Recall,POSITIVE
997,sais,80,Know,MIXED
998,madame,80,Madam.,NEGATIVE


In [11]:
# associer le mot à son embedding

df["embedding"] = df["en"].apply(lambda x: embeddings_dict.get(x, np.zeros(50)))

In [13]:
words = pd.read_csv("most_frequent_words.csv")["en"]
new_words = []
for word in words:
    if type(word) == str:
        new_words += re.sub(r"[^a-zA-Z ]+", "", word.strip()).lower().split(" ")

new_words = list(set(new_words) - set([""]))

new_words = pd.DataFrame(new_words, columns=["en"])

In [21]:

new_words["embedding"] = new_words["en"].apply(lambda x: embeddings_dict.get(x, '-1'))

In [30]:
# afficher les -1
new_words = new_words[new_words["embedding"] != '-1']

/Users/victor/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/pandas/core/ops/array_ops.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


In [32]:
new_words.to_csv("wordsmost_frequent_with_vectors.csv", index=False)

## Tensorboard

In [25]:
new_words

,en,embedding
0,advisers,"[0.83158, 0.15111, 0.65911, -0.35074, 0.60972,..."
1,former,"[-0.53526, 0.54543, 0.11158, 0.59775, 0.30465,..."
2,taken,"[0.89173, -0.20309, -0.070216, -0.59429, 0.352..."
3,easily,"[0.22003, -0.72782, 1.1417, -0.04534, 0.1448, ..."
4,line,"[-0.22258, 0.089765, 0.45857, -0.31071, -0.169..."
...,...,...
679,money,"[0.59784, -0.057026, 0.97746, -0.58504, 0.3738..."
680,secretary,"[-0.14508, 0.14645, -0.76128, 0.38553, 0.74353..."
681,fine,"[-0.70315, 0.43281, -0.94625, -0.39258, 0.5245..."
682,avette,-1


In [36]:
# convertir la colonne 'en' en tsv metadata.tsv 
new_words["en"].to_csv("metadata.tsv", sep="\t", index=False, header=False)


In [ ]:

# convertir la colonne 'embedding' en tsv vectors.tsv de ce format 
"""
0.1\t0.2\t0.5\t0.9
0.2\t0.1\t5.0\t0.2
0.4\t0.1\t7.0\t0.8
"""


new_words["embedding"].apply(lambda x: "\t".join([str(i) for i in x])).to_csv("vectors.tsv", sep="\t", index=False, header=False)


In [35]:

# Spécifiez le chemin du fichier TSV de sortie
output_file_path = 'vectors.tsv'

# Écrire le fichier TSV avec le format souhaité
with open(output_file_path, 'w') as file:
    # Écrire l'en-tête (si nécessaire)
    # file.write('\t'.join([f'Dim{i}' for i in range(len(df['embeddings'][0]))]) + '\n')

    # Écrire les vecteurs dans le format spécifié
    for vector in new_words['embedding']:
        file.write('\t'.join(map(str, vector)) + '\n')

